# Basic Usage

In this notebook we will show how to use the 'ngffImage' class to manage a OME-NGFF image.

For this example we will use a small example image that can be downloaded from the following link: [example ome-zarr](https://zenodo.org/records/13305156)

You can download the example image (on Linux and Mac os) by running the following command:

```bash
bash setup_data.sh
```
from the root of the repository.

## NgffImage

The `NgffImage` provides a high-level interface to read, write and manipulate NGFF images.
A `NgffImage` can be created from a storelike object (e.g. a path to a directory or a zarr store) or from a `zarr.Group` object. 

In [ ]:
from ngio.core import NgffImage
ngff_image = NgffImage("../../data/20200812-CardiomyocyteDifferentiation14-Cycle1_mip.zarr/B/03/0")

The `ngff_image` contains several attributes and methods to interact with the `OME-NGFF` (`OME-Zarr`) file on the storage.

In [ ]:
# Explore object metadata
print("Levels: ", ngff_image.levels_paths)
print("Num Levels: ", ngff_image.num_levels)

Get a single `level` of the image pyramid as `Image` (to know more about the `Image` class, please refer to the [Image notebook](./image.ipynb)
The `Image` object is the main object to interact with the image. 
It contains methods to interact with the image data and metadata.

In [ ]:
from ngio.ngff_meta import PixelSize

# 1. Get image from highest resolution (default)
image = ngff_image.get_image()
print(f"{image.shape=}")
print(f"{image.pixel_size=}")

# 2. Get image from a specific level using the path keyword
print(f"{image.shape=}")
print(f"{image.pixel_size=}")

# 3. Get image from a specific pixel size using the pixel_size keyword
image = ngff_image.get_image(pixel_size=PixelSize(x=0.65, y=0.65, z=1))
print(f"{image.shape=}")
print(f"{image.pixel_size=}")

## Labels

The `NgffImage` can also be used to load labels from a `OME-NGFF` file.

In [ ]:
print("List of Labels: ", ngff_image.label.list())

label_nuclei = ngff_image.label.get_label("nuclei", path="0")
print(f"{label_nuclei.shape=}")
print(f"{label_nuclei.pixel_size=}")

## Tables

The `NgffImage` can also be used to load tables from a `OME-NGFF` file.

`ngio` supports three types of tables:
 - `features table` A simple table to store features associated with a label.
 - `roi table` A table to store regions of interest.
 - `masking roi tables` A table to store single objects bounding boxes associated with a label.

In [ ]:
print("List of Tables: ", ngff_image.table.list())
print(" - Feature tables: ", ngff_image.table.list(table_type='feature_table'))
print(" - Roi tables: ", ngff_image.table.list(table_type='roi_table'))
print(" - Masking Roi tables: ", ngff_image.table.list(table_type='masking_roi_table'))

In [ ]:
# Loading a table
feature_table = ngff_image.table.get_table("regionprops_DAPI")
feature_table.table

In [ ]:
# Loading a roi table
roi_table = ngff_image.table.get_table("FOV_ROI_table")

print(f"{roi_table.field_indexes=}")
print(f"{roi_table.get_roi('FOV_1')=}")

roi_table.table

## Derive a new NgffImage

When processing an image, it is often useful to derive a new image from the original image.
The `NgffImage` class provides a method to derive a new image from the original image.
When deriving a new image, a new `NgffImage` object is created with the same metadata as the original image. Optionally the 
user can specify different metadata for the new image(.e.g. different channels names).

In [ ]:
new_ngff_image = ngff_image.derive_new_image("../../data/new_ome.zarr", name="new_image")

print(f"{new_ngff_image.store=}")
print(f"{new_ngff_image.levels_paths=}")
print(f"{new_ngff_image.num_levels=}")